In [101]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input

In [103]:
dataPathLocal = "G:/My Drive/keras_project/dataDigits"

In [105]:
batch_size = 32
num_classes = 53

# preserves 16:9 aspect-ratio of images taken
img_height = 64
img_width = 64


In [107]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    dataPathLocal,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(img_height, img_width),
     batch_size=batch_size
)

Found 25757 files belonging to 53 classes.
Using 20606 files for training.
Using 5151 files for validation.


In [113]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [115]:
model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, name="outputs")
])

In [117]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [119]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 53)             │         6,837 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 571,349 (2.18 MB)

 Trainable params: 571,349 (2.18 MB)

 Non-trainable params: 0 (0.00 B)

In [121]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
644/644 ━━━━━━━━━━━━━━━━━━━━ 165s 68ms/step - accuracy: 0.2123 - loss: 2.9944 - val_accuracy: 0.7781 - val_loss: 0.7075
Epoch 2/5
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.8423 - loss: 0.4898 - val_accuracy: 0.9466 - val_loss: 0.1733
Epoch 3/5
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.9437 - loss: 0.1617 - val_accuracy: 0.9505 - val_loss: 0.1565
Epoch 4/5
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9694 - loss: 0.0903 - val_accuracy: 0.9816 - val_loss: 0.0560
Epoch 5/5
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.9764 - loss: 0.0710 - val_accuracy: 0.9893 - val_loss: 0.0338


In [123]:
card_names = "kaiserin,koenig,koenigin,kriegsherr,prinzessin,elbenschuetzen,leichte_kavallerie,ritter,waldlaeufer,zwergeninfanterie,buch_der_veraenderung,juwel_der_ordnung,rune_des_schutzes,schild_von_keth,weltenbaum,basilisk,drache,einhorn,hydra,schlachtross,blitz,buschfeuer,feuerwesen,kerze,schmiede,grosse_flut,insel,quelle_des_lebens,sumpf,wasserwesen,erdwesen,gebirge,glockenturm,hoehle,wald,elbischer_bogen,kampfzeppelin,kriegsschiff,schwert_von_keth,zauberstab,blizzard,luftwesen,rauch,regensturm,wirbelsturm,hexenmeister,sammler,herr_der_bestien,totenbeschwoerer,magierin,doppelgaenger,gestaltenwandler,spiegelung"
card_names_sep = card_names.split(",")
card_names_sep.sort()

In [135]:
path = "G:/My Drive/keras_project/validation/koenig1.jpg"

img = tf.keras.utils.load_img(
    path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
pos = np.argmax(score)
prediction = card_names_sep[pos]


print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(prediction, 100 * np.max(score))
)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_4" is incompatible with the layer: expected axis -1 of input shape to have value 56320, but received input with shape (1, 4096)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 64, 64, 3), dtype=float32)
  • training=False
  • mask=None

In [131]:
model.save("C:/Users/Test/Desktop/WiSe24/nn/project/model/model_small.keras")

In [ ]:
model = keras.models.load_model("C:/Users/Test/Desktop/WiSe24/nn/project/model/realms.keras")

In [133]:
import tensorflow as tf

model = tf.keras.models.load_model("C:/Users/Test/Desktop/WiSe24/nn/project/model/model_small.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_small.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Test\AppData\Local\Temp\tmpto0s_khw\assets


INFO:tensorflow:Assets written to: C:\Users\Test\AppData\Local\Temp\tmpto0s_khw\assets


Saved artifact at 'C:\Users\Test\AppData\Local\Temp\tmpto0s_khw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 320, 180, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 53), dtype=tf.float32, name=None)
Captures:
  3058394933648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394927696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394926736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394927312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394923280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394929616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394928656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394930192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394925776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3058394936720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  305839